In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import ast

In [ ]:
declaracoes = pd.read_csv("declaracoes-nlp-new.csv", converters={'entities': lambda x: x[1:-1].split(',')})
declaracoes

In [3]:
declaracoes_nopunct = pd.read_csv("declaracoes-nlp-no-punct.csv", converters={'tokens': lambda x: x[1:-1].split(',')})
declaracoes_nopunct

,speaker,partido,speech,filename,no_punct,tokens,lemma,entities
0,Ana Mesquita,PCP,"Sr. Presidente, Sr.as Deputadas e Srs. Deputad...",darl13sl03n050.txt,Sr Presidente Sras Deputadas e Srs Deputados O...,"['Sr', 'Presidente', 'Sras', 'Deputadas', ...","['Sr', 'Presidente', 'Sras', 'Deputadas', 'e',...","[(Sr Presidente Sras Deputadas, 'PER', 4317129..."
1,Ana Mesquita,PCP,A esmagadora maioria da população aprendeu a n...,darl13sl03n050.txt,A esmagadora maioria da população aprendeu a n...,"['A', 'esmagadora', 'maioria', 'da', 'popu...","['A', 'esmagador', 'maioria', 'da', 'população...","[(Acordo É preferível, 'MISC', 765424194013315..."
2,Ana Mesquita,PCP,"Já houve um, na Legislatura anterior!",darl13sl03n050.txt,Já houve um na Legislatura anterior,"['Já', 'houve', 'um', 'na', 'Legislatura',...","['Já', 'haver', 'um', 'o', 'Legislatura', 'ant...","[(Legislatura, 'MISC', 7654241940133152407)]"
3,Ana Mesquita,PCP,Esse argumento é falso!,darl13sl03n050.txt,Esse argumento é falso,"['Esse', 'argumento', 'é', 'falso']","['Esse', 'argumentar', 'ser', 'falsar']",[]
4,João Oliveira,PCP,Exatamente!,darl13sl03n050.txt,Exatamente,['Exatamente'],['Exatamente'],[]
...,...,...,...,...,...,...,...,...
92233,Carla Cruz,PCP,"Sr. Presidente, Srs. Deputados: Só há melhor s...",darl13sl03n058.txt,Sr Presidente Srs Deputados Só há melhor saúde...,"['Sr', 'Presidente', 'Srs', 'Deputados', '...","['Sr', 'Presidente', 'Srs', 'Deputados', 'Só',...","[(Sr Presidente Srs Deputados, 'PER', 43171290..."
92234,Carla Cruz,PCP,É verdade!,darl13sl03n058.txt,É verdade,"['É', 'verdade']","['É', 'verdade']",[]
92235,Carla Cruz,PCP,"Sr. Presidente, Srs. Deputados: O CDS e o PSD ...",darl13sl03n058.txt,Sr Presidente Srs Deputados O CDS e o PSD só d...,"['Sr', 'Presidente', 'Srs', 'Deputados', '...","['Sr', 'Presidente', 'Srs', 'Deputados', 'O', ...","[(Sr Presidente Srs Deputados, 'PER', 43171290..."
92236,Carla Cruz,PCP,Foi essa a política que fizeram ao longo do vo...,darl13sl03n058.txt,Foi essa a política que fizeram ao longo do vo...,"['Foi', 'essa', 'a', 'política', 'que', '...","['Foi', 'esse', 'o', 'político', 'que', 'fazer...","[(Aplausos do PCP, 'MISC', 7654241940133152407..."


In [5]:
#token frequency by party

def flatten(t):
    return [item for sublist in t for item in sublist]

counter_by_party = {}

for party in list(set(declaracoes_nopunct["partido"])):
    party_rows = declaracoes_nopunct.loc[declaracoes_nopunct["partido"] == party]
    party_tokens = flatten(party_rows["tokens"])
    counter = [Counter(party_tokens)]

    counter_by_party[party] = counter
    
counter_by_party

{'PCP': [Counter({"'Sr'": 4116,
           " 'Presidente'": 6601,
           " 'Sras'": 1710,
           " 'Deputadas'": 79,
           " 'e'": 62838,
           " 'Srs'": 5475,
           " 'Deputados'": 4995,
           " 'O'": 4910,
           " 'PCP'": 12243,
           " 'saúda'": 44,
           " 'os'": 18315,
           " 'mais'": 7761,
           " 'de'": 87520,
           " '20'": 373,
           " '000'": 438,
           " 'subscritores'": 95,
           " 'da'": 27966,
           " 'petição'": 438,
           " 'alguns'": 532,
           " 'dos'": 15677,
           " 'quais'": 571,
           " 'aqui'": 3000,
           " 'presentes'": 145,
           " 'nas'": 2566,
           " 'galerias'": 56,
           " 'Assembleia'": 1946,
           " 'República'": 1808,
           " 'Em'": 763,
           " '4'": 238,
           " 'Junho'": 1,
           " '1991'": 7,
           " 'o'": 46185,
           " 'foi'": 4175,
           " 'único'": 224,
           " 'Grupo'": 600,
       

In [51]:
tokens_by_party = pd.concat({k: pd.DataFrame(v).T for k, v in counter_by_party.items()}, axis=0)
tokens_by_party = tokens_by_party.stack().unstack(level=1)
tokens_by_party = tokens_by_party.transpose()
tokens_by_party = tokens_by_party.fillna(0)
tokens_by_party = tokens_by_party.astype("int") 
tokens_by_party = tokens_by_party.reset_index()
tokens_by_party = tokens_by_party.droplevel(1, axis=1)

,index,PCP,PSD,CH,N insc,BE,PS,PAN,PEV,IL,CDS-PP
,,0,0,0,0,0,0,0,0,0,0
0,' ',0,1,0,0,0,0,0,0,0,1
1,' ',6,14,1,0,4,9,2,2,2,11
2,' ',3072,3943,262,0,2889,3715,831,1153,197,2946
3,' ',7065,12351,823,2,8671,10246,1677,2735,656,11893
4,'0',6,9,0,0,9,9,4,10,2,9
...,...,...,...,...,...,...,...,...,...,...,...
97972,'Ótimo',0,2,0,0,0,0,0,0,0,0
97973,'Ótimos',0,0,0,0,1,0,0,0,0,0
97974,'Último',0,0,0,0,0,1,0,0,0,0


,index,PCP,PSD,CH,N insc,BE,PS,PAN,PEV,IL,CDS-PP
0,' ',0,1,0,0,0,0,0,0,0,1
1,' ',6,14,1,0,4,9,2,2,2,11
2,' ',3072,3943,262,0,2889,3715,831,1153,197,2946
3,' ',7065,12351,823,2,8671,10246,1677,2735,656,11893
4,'0',6,9,0,0,9,9,4,10,2,9
...,...,...,...,...,...,...,...,...,...,...,...
97972,'Ótimo',0,2,0,0,0,0,0,0,0,0
97973,'Ótimos',0,0,0,0,1,0,0,0,0,0
97974,'Último',0,0,0,0,0,1,0,0,0,0
97975,'às',0,0,0,0,0,0,0,0,0,1


In [ ]:
'''entities = declaracoes[["filename", "partido", "entities"]]

counter = {}
counter_party = {}

for party in list(set( entities["partido"] )):
    counter_party.setdefault(party, {})

    for ent_list in entities["entities"]:
        list_of_all_items  = str(ent_list).replace("[", "").replace("]", "").split(", ")
        grouped_items = np.array_split(list_of_all_items,3)
        stringified_groups = [", ".join(x) for x in grouped_items]
        
        for ent in stringified_groups:

            print("ent:")
            print((ent))
            # counter_party[party].setdefault(ent, 0)
            # counter_party[party][ent] += 1


    # counter_party = 1'''


'''for lst in entities["entities"]:
        for tup in lst:
          tup = tup.replace('(','')
          tup = tup.replace(')','')
          try:
            counter[tup] +=1
          except KeyError:
            counter[tup] = 1'''


#counter_party

In [ ]:
'''entities = declaracoes[["filename", "partido", "entities"]]

counter = {}
counter_party = 0

for party in entities["partido"]:
    counter['partido'] = party
    counter_party = +1

for fake_string in entities["entities"]:
    fake_string = fake_string.replace('[','')
    fake_string = fake_string.replace('(','')
    fake_string = fake_string.replace(')','')
    fake_string = fake_string.replace(']','')
    fake_string = fake_string.replace('\'','')
    fake_string = fake_string.replace('\"','')
    fake_string = fake_string.split(',')
    
    for name in fake_string:
        name = name.lstrip(" ")
        try:
            counter[name] +=1
        except KeyError:
            counter[name] = 1

counter'''
